In [1]:
import numpy as np
import pandas as pd
import os
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from scipy.interpolate import UnivariateSpline
import statsmodels.api as sm
import matplotlib.pyplot as plt
import collections
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

pd.set_option("display.max_column", 999)

In [2]:
tourney_results = pd.concat([
    pd.read_csv("MNCAATourneyDetailedResults.csv"),
    pd.read_csv("WNCAATourneyDetailedResults.csv"),
], ignore_index=True)

seeds = pd.concat([
    pd.read_csv("MNCAATourneySeeds.csv"),
    pd.read_csv("WNCAATourneySeeds.csv"),
], ignore_index=True)

regular_results = pd.concat([
    pd.read_csv("MRegularSeasonDetailedResults.csv"),
    pd.read_csv("WRegularSeasonDetailedResults.csv"),
], ignore_index=True)

In [3]:
tourney_results.shape

(2075, 34)

In [4]:
seeds.shape

(4098, 3)

In [5]:
regular_results.shape

(184024, 34)

In [6]:
regular_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR',
       'WAst', 'WTO', 'WStl', 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3',
       'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF'],
      dtype='object')

In [7]:
regular_results_swap = regular_results[[
    'Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

In [8]:
regular_results.head()

Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT  WFGM  WFGA  \
0    2003      10     1104      68     1328      62    N      0    27    58   
1    2003      10     1272      70     1393      63    N      0    26    62   
2    2003      11     1266      73     1437      61    N      0    24    58   
3    2003      11     1296      56     1457      50    N      0    18    38   
4    2003      11     1400      77     1208      71    N      0    30    61   

   WFGM3  WFGA3  WFTM  WFTA  WOR  WDR  WAst  WTO  WStl  WBlk  WPF  LFGM  LFGA  \
0      3     14    11    18   14   24    13   23     7     1   22    22    53   
1      8     20    10    19   15   28    16   13     4     4   18    24    67   
2      8     18    17    29   17   26    15   10     5     2   25    22    73   
3      3      9    17    31    6   19    11   12    14     2   18    18    49   
4      6     14    11    13   17   22    12   14     4     4   20    24    62   

   LFGM3  LFGA3  LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk  LPF  
0      2     10    16    22   10   22     8   18     9     2   20  
1      6     24     9    20   20   25     7   12     8     6   16  
2      3     26    14    23   31   22     9   12     2     5   23  
3      6     22     8    15   17   20     9   19     4     3   23  
4      6     16    17    27   21   15    12   10     7     1   14

In [9]:
regular_results_swap.head()

Season  DayNum  LTeamID  LScore  WTeamID  WScore WLoc  NumOT  LFGM  LFGA  \
0    2003      10     1328      62     1104      68    N      0    22    53   
1    2003      10     1393      63     1272      70    N      0    24    67   
2    2003      11     1437      61     1266      73    N      0    22    73   
3    2003      11     1457      50     1296      56    N      0    18    49   
4    2003      11     1208      71     1400      77    N      0    24    62   

   LFGM3  LFGA3  LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk  LPF  WFGM  WFGA  \
0      2     10    16    22   10   22     8   18     9     2   20    27    58   
1      6     24     9    20   20   25     7   12     8     6   16    26    62   
2      3     26    14    23   31   22     9   12     2     5   23    24    58   
3      6     22     8    15   17   20     9   19     4     3   23    18    38   
4      6     16    17    27   21   15    12   10     7     1   14    30    61   

   WFGM3  WFGA3  WFTM  WFTA  WOR  WDR  WAst  WTO  WStl  WBlk  WPF  
0      3     14    11    18   14   24    13   23     7     1   22  
1      8     20    10    19   15   28    16   13     4     4   18  
2      8     18    17    29   17   26    15   10     5     2   25  
3      3      9    17    31    6   19    11   12    14     2   18  
4      6     14    11    13   17   22    12   14     4     4   20

In [10]:
regular_results_swap.loc[regular_results['WLoc'] == 'H', 'WLoc'] = 'A'
regular_results_swap.loc[regular_results['WLoc'] == 'A', 'WLoc'] = 'H'
regular_results.columns.values[6] = 'location'
regular_results_swap.columns.values[6] = 'location'

In [11]:
regular_results.head()

Season  DayNum  WTeamID  WScore  LTeamID  LScore location  NumOT  WFGM  \
0    2003      10     1104      68     1328      62        N      0    27   
1    2003      10     1272      70     1393      63        N      0    26   
2    2003      11     1266      73     1437      61        N      0    24   
3    2003      11     1296      56     1457      50        N      0    18   
4    2003      11     1400      77     1208      71        N      0    30   

   WFGA  WFGM3  WFGA3  WFTM  WFTA  WOR  WDR  WAst  WTO  WStl  WBlk  WPF  LFGM  \
0    58      3     14    11    18   14   24    13   23     7     1   22    22   
1    62      8     20    10    19   15   28    16   13     4     4   18    24   
2    58      8     18    17    29   17   26    15   10     5     2   25    22   
3    38      3      9    17    31    6   19    11   12    14     2   18    18   
4    61      6     14    11    13   17   22    12   14     4     4   20    24   

   LFGA  LFGM3  LFGA3  LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk  LPF  
0    53      2     10    16    22   10   22     8   18     9     2   20  
1    67      6     24     9    20   20   25     7   12     8     6   16  
2    73      3     26    14    23   31   22     9   12     2     5   23  
3    49      6     22     8    15   17   20     9   19     4     3   23  
4    62      6     16    17    27   21   15    12   10     7     1   14

Swapping here so the model doesnt know

In [12]:
regular_results_swap.head()

Season  DayNum  LTeamID  LScore  WTeamID  WScore location  NumOT  LFGM  \
0    2003      10     1328      62     1104      68        N      0    22   
1    2003      10     1393      63     1272      70        N      0    24   
2    2003      11     1437      61     1266      73        N      0    22   
3    2003      11     1457      50     1296      56        N      0    18   
4    2003      11     1208      71     1400      77        N      0    24   

   LFGA  LFGM3  LFGA3  LFTM  LFTA  LOR  LDR  LAst  LTO  LStl  LBlk  LPF  WFGM  \
0    53      2     10    16    22   10   22     8   18     9     2   20    27   
1    67      6     24     9    20   20   25     7   12     8     6   16    26   
2    73      3     26    14    23   31   22     9   12     2     5   23    24   
3    49      6     22     8    15   17   20     9   19     4     3   23    18   
4    62      6     16    17    27   21   15    12   10     7     1   14    30   

   WFGA  WFGM3  WFGA3  WFTM  WFTA  WOR  WDR  WAst  WTO  WStl  WBlk  WPF  
0    58      3     14    11    18   14   24    13   23     7     1   22  
1    62      8     20    10    19   15   28    16   13     4     4   18  
2    58      8     18    17    29   17   26    15   10     5     2   25  
3    38      3      9    17    31    6   19    11   12    14     2   18  
4    61      6     14    11    13   17   22    12   14     4     4   20

In [13]:
regular_results.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(regular_results.columns)]
regular_results_swap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(regular_results.columns)]

In [14]:
regular_data = pd.concat([regular_results, regular_results_swap]).sort_index().reset_index(drop = True)

Replacing with T1 and T2 so their is no indication of who is winning 

In [15]:
regular_data.head(10)

Season  DayNum  T1_TeamID  T1_Score  T2_TeamID  T2_Score location  NumOT  \
0    2003      10       1104        68       1328        62        N      0   
1    2003      10       1328        62       1104        68        N      0   
2    2003      10       1393        63       1272        70        N      0   
3    2003      10       1272        70       1393        63        N      0   
4    2003      11       1266        73       1437        61        N      0   
5    2003      11       1437        61       1266        73        N      0   
6    2003      11       1296        56       1457        50        N      0   
7    2003      11       1457        50       1296        56        N      0   
8    2003      11       1400        77       1208        71        N      0   
9    2003      11       1208        71       1400        77        N      0   

   T1_FGM  T1_FGA  T1_FGM3  T1_FGA3  T1_FTM  T1_FTA  T1_OR  T1_DR  T1_Ast  \
0      27      58        3       14      11      18     14     24      13   
1      22      53        2       10      16      22     10     22       8   
2      24      67        6       24       9      20     20     25       7   
3      26      62        8       20      10      19     15     28      16   
4      24      58        8       18      17      29     17     26      15   
5      22      73        3       26      14      23     31     22       9   
6      18      38        3        9      17      31      6     19      11   
7      18      49        6       22       8      15     17     20       9   
8      30      61        6       14      11      13     17     22      12   
9      24      62        6       16      17      27     21     15      12   

   T1_TO  T1_Stl  T1_Blk  T1_PF  T2_FGM  T2_FGA  T2_FGM3  T2_FGA3  T2_FTM  \
0     23       7       1     22      22      53        2       10      16   
1     18       9       2     20      27      58        3       14      11   
2     12       8       6     16      26      62        8       20      10   
3     13       4       4     18      24      67        6       24       9   
4     10       5       2     25      22      73        3       26      14   
5     12       2       5     23      24      58        8       18      17   
6     12      14       2     18      18      49        6       22       8   
7     19       4       3     23      18      38        3        9      17   
8     14       4       4     20      24      62        6       16      17   
9     10       7       1     14      30      61        6       14      11   

   T2_FTA  T2_OR  T2_DR  T2_Ast  T2_TO  T2_Stl  T2_Blk  T2_PF  
0      22     10     22       8     18       9       2     20  
1      18     14     24      13     23       7       1     22  
2      19     15     28      16     13       4       4     18  
3      20     20     25       7     12       8       6     16  
4      23     31     22       9     12       2       5     23  
5      29     17     26      15     10       5       2     25  
6      15     17     20       9     19       4       3     23  
7      31      6     19      11     12      14       2     18  
8      27     21     15      12     10       7       1     14  
9      13     17     22      12     14       4       4     20

In [16]:
tourney_results = pd.concat([
    pd.read_csv("MNCAATourneyDetailedResults.csv"),
    pd.read_csv("WNCAATourneyDetailedResults.csv"),
], ignore_index=True)

seeds = pd.concat([
    pd.read_csv("MNCAATourneySeeds.csv"),
    pd.read_csv("WNCAATourneySeeds.csv"),
], ignore_index=True)

regular_results = pd.concat([
    pd.read_csv("MRegularSeasonDetailedResults.csv"),
    pd.read_csv("WRegularSeasonDetailedResults.csv"),
], ignore_index=True)

In [17]:
def prepare_data(df):
    dfswap = df[['Season', 'DayNum', 'LTeamID', 'LScore', 'WTeamID', 'WScore', 'WLoc', 'NumOT', 
    'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl', 'LBlk', 'LPF', 
    'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl', 'WBlk', 'WPF']]

    dfswap.loc[df['WLoc'] == 'H', 'WLoc'] = 'A'
    dfswap.loc[df['WLoc'] == 'A', 'WLoc'] = 'H'
    df.columns.values[6] = 'location'
    dfswap.columns.values[6] = 'location'    
      
    df.columns = [x.replace('W','T1_').replace('L','T2_') for x in list(df.columns)]
    dfswap.columns = [x.replace('L','T1_').replace('W','T2_') for x in list(dfswap.columns)]

    output = pd.concat([df, dfswap]).reset_index(drop=True)
    output.loc[output.location=='N','location'] = '0'
    output.loc[output.location=='H','location'] = '1'
    output.loc[output.location=='A','location'] = '-1'
    output.location = output.location.astype(int)
    
    output['PointDiff'] = output['T1_Score'] - output['T2_Score']
    
    return output

In [18]:
regular_data = prepare_data(regular_results)
tourney_data = prepare_data(tourney_results)

In [19]:
regular_data.shape

(368048, 35)

In [20]:
tourney_data.shape

(4150, 35)

In [21]:
tourney_data.columns

Index(['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID', 'T2_Score',
       'location', 'NumOT', 'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM',
       'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk',
       'T1_PF', 'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA',
       'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF',
       'PointDiff'],
      dtype='object')

Getting Mean from season for below this 

In [22]:
boxscore_cols = ['T1_Score', 'T2_Score', 
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_FTM', 'T1_FTA', 'T1_OR', 'T1_DR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_Blk', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_FTM', 'T2_FTA', 'T2_OR', 'T2_DR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk', 'T2_PF', 
        'PointDiff']

boxscore_cols = [
        'T1_FGM', 'T1_FGA', 'T1_FGM3', 'T1_FGA3', 'T1_OR', 'T1_Ast', 'T1_TO', 'T1_Stl', 'T1_PF', 
        'T2_FGM', 'T2_FGA', 'T2_FGM3', 'T2_FGA3', 'T2_OR', 'T2_Ast', 'T2_TO', 'T2_Stl', 'T2_Blk',  
        'PointDiff']

funcs = [np.mean]

In [23]:
season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs)
season_statistics.head()

T1_FGM     T1_FGA   T1_FGM3    T1_FGA3      T1_OR  \
                       mean       mean      mean       mean       mean   
Season T1_TeamID                                                         
2003   1102       19.142857  39.785714  7.821429  20.821429   4.178571   
       1103       27.148148  55.851852  5.444444  16.074074   9.777778   
       1104       24.035714  57.178571  6.357143  19.857143  13.571429   
       1105       24.384615  61.615385  7.576923  20.769231  13.500000   
       1106       23.428571  55.285714  6.107143  17.642857  12.285714   

                     T1_Ast      T1_TO    T1_Stl      T1_PF     T2_FGM  \
                       mean       mean      mean       mean       mean   
Season T1_TeamID                                                         
2003   1102       13.000000  11.428571  5.964286  18.750000  19.285714   
       1103       15.222222  12.629630  7.259259  19.851852  27.777778   
       1104       12.107143  13.285714  6.607143  18.035714  23.250000   
       1105       14.538462  18.653846  9.307692  20.230769  27.000000   
       1106       11.678571  17.035714  8.357143  18.178571  21.714286   

                     T2_FGA   T2_FGM3    T2_FGA3      T2_OR     T2_Ast  \
                       mean      mean       mean       mean       mean   
Season T1_TeamID                                                         
2003   1102       42.428571  4.750000  12.428571   9.607143   9.142857   
       1103       57.000000  6.666667  18.370370  12.037037  15.481481   
       1104       55.500000  6.357143  19.142857  10.892857  11.678571   
       1105       58.961538  6.269231  17.538462  13.192308  15.807692   
       1106       53.392857  4.785714  15.214286  11.321429  11.785714   

                      T2_TO    T2_Stl    T2_Blk PointDiff  
                       mean      mean      mean      mean  
Season T1_TeamID                                           
2003   1102       12.964286  5.428571  1.571429  0.250000  
       1103       15.333333  6.407407  2.851852  0.629630  
       1104       13.857143  5.535714  3.178571  4.285714  
       1105       18.807692  9.384615  4.192308 -4.884615  
       1106       15.071429  8.785714  3.178571 -0.142857

In [24]:
season_statistics = regular_data.groupby(["Season", 'T1_TeamID'])[boxscore_cols].agg(funcs).reset_index()
season_statistics.head()

Season T1_TeamID     T1_FGM     T1_FGA   T1_FGM3    T1_FGA3      T1_OR  \
                         mean       mean      mean       mean       mean   
0   2003      1102  19.142857  39.785714  7.821429  20.821429   4.178571   
1   2003      1103  27.148148  55.851852  5.444444  16.074074   9.777778   
2   2003      1104  24.035714  57.178571  6.357143  19.857143  13.571429   
3   2003      1105  24.384615  61.615385  7.576923  20.769231  13.500000   
4   2003      1106  23.428571  55.285714  6.107143  17.642857  12.285714   

      T1_Ast      T1_TO    T1_Stl      T1_PF     T2_FGM     T2_FGA   T2_FGM3  \
        mean       mean      mean       mean       mean       mean      mean   
0  13.000000  11.428571  5.964286  18.750000  19.285714  42.428571  4.750000   
1  15.222222  12.629630  7.259259  19.851852  27.777778  57.000000  6.666667   
2  12.107143  13.285714  6.607143  18.035714  23.250000  55.500000  6.357143   
3  14.538462  18.653846  9.307692  20.230769  27.000000  58.961538  6.269231   
4  11.678571  17.035714  8.357143  18.178571  21.714286  53.392857  4.785714   

     T2_FGA3      T2_OR     T2_Ast      T2_TO    T2_Stl    T2_Blk PointDiff  
        mean       mean       mean       mean      mean      mean      mean  
0  12.428571   9.607143   9.142857  12.964286  5.428571  1.571429  0.250000  
1  18.370370  12.037037  15.481481  15.333333  6.407407  2.851852  0.629630  
2  19.142857  10.892857  11.678571  13.857143  5.535714  3.178571  4.285714  
3  17.538462  13.192308  15.807692  18.807692  9.384615  4.192308 -4.884615  
4  15.214286  11.321429  11.785714  15.071429  8.785714  3.178571 -0.142857

In [25]:
season_statistics.columns = [''.join(col).strip() for col in season_statistics.columns.values]
season_statistics.head()

Season  T1_TeamID  T1_FGMmean  T1_FGAmean  T1_FGM3mean  T1_FGA3mean  \
0    2003       1102   19.142857   39.785714     7.821429    20.821429   
1    2003       1103   27.148148   55.851852     5.444444    16.074074   
2    2003       1104   24.035714   57.178571     6.357143    19.857143   
3    2003       1105   24.384615   61.615385     7.576923    20.769231   
4    2003       1106   23.428571   55.285714     6.107143    17.642857   

   T1_ORmean  T1_Astmean  T1_TOmean  T1_Stlmean  T1_PFmean  T2_FGMmean  \
0   4.178571   13.000000  11.428571    5.964286  18.750000   19.285714   
1   9.777778   15.222222  12.629630    7.259259  19.851852   27.777778   
2  13.571429   12.107143  13.285714    6.607143  18.035714   23.250000   
3  13.500000   14.538462  18.653846    9.307692  20.230769   27.000000   
4  12.285714   11.678571  17.035714    8.357143  18.178571   21.714286   

   T2_FGAmean  T2_FGM3mean  T2_FGA3mean  T2_ORmean  T2_Astmean  T2_TOmean  \
0   42.428571     4.750000    12.428571   9.607143    9.142857  12.964286   
1   57.000000     6.666667    18.370370  12.037037   15.481481  15.333333   
2   55.500000     6.357143    19.142857  10.892857   11.678571  13.857143   
3   58.961538     6.269231    17.538462  13.192308   15.807692  18.807692   
4   53.392857     4.785714    15.214286  11.321429   11.785714  15.071429   

   T2_Stlmean  T2_Blkmean  PointDiffmean  
0    5.428571    1.571429       0.250000  
1    6.407407    2.851852       0.629630  
2    5.535714    3.178571       4.285714  
3    9.384615    4.192308      -4.884615  
4    8.785714    3.178571      -0.142857

## Breaking down into Team 1 and Team 2

In [26]:
season_statistics_T1 = season_statistics.copy()
season_statistics_T2 = season_statistics.copy()

season_statistics_T1.columns = ["T1_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T1.columns)]
season_statistics_T2.columns = ["T2_" + x.replace("T1_","").replace("T2_","opponent_") for x in list(season_statistics_T2.columns)]
season_statistics_T1.columns.values[0] = "Season"
season_statistics_T2.columns.values[0] = "Season"

In [27]:
season_statistics_T1.head()

Season  T1_TeamID  T1_FGMmean  T1_FGAmean  T1_FGM3mean  T1_FGA3mean  \
0    2003       1102   19.142857   39.785714     7.821429    20.821429   
1    2003       1103   27.148148   55.851852     5.444444    16.074074   
2    2003       1104   24.035714   57.178571     6.357143    19.857143   
3    2003       1105   24.384615   61.615385     7.576923    20.769231   
4    2003       1106   23.428571   55.285714     6.107143    17.642857   

   T1_ORmean  T1_Astmean  T1_TOmean  T1_Stlmean  T1_PFmean  \
0   4.178571   13.000000  11.428571    5.964286  18.750000   
1   9.777778   15.222222  12.629630    7.259259  19.851852   
2  13.571429   12.107143  13.285714    6.607143  18.035714   
3  13.500000   14.538462  18.653846    9.307692  20.230769   
4  12.285714   11.678571  17.035714    8.357143  18.178571   

   T1_opponent_FGMmean  T1_opponent_FGAmean  T1_opponent_FGM3mean  \
0            19.285714            42.428571              4.750000   
1            27.777778            57.000000              6.666667   
2            23.250000            55.500000              6.357143   
3            27.000000            58.961538              6.269231   
4            21.714286            53.392857              4.785714   

   T1_opponent_FGA3mean  T1_opponent_ORmean  T1_opponent_Astmean  \
0             12.428571            9.607143             9.142857   
1             18.370370           12.037037            15.481481   
2             19.142857           10.892857            11.678571   
3             17.538462           13.192308            15.807692   
4             15.214286           11.321429            11.785714   

   T1_opponent_TOmean  T1_opponent_Stlmean  T1_opponent_Blkmean  \
0           12.964286             5.428571             1.571429   
1           15.333333             6.407407             2.851852   
2           13.857143             5.535714             3.178571   
3           18.807692             9.384615             4.192308   
4           15.071429             8.785714             3.178571   

   T1_PointDiffmean  
0          0.250000  
1          0.629630  
2          4.285714  
3         -4.884615  
4         -0.142857

In [28]:
season_statistics_T2.head()

Season  T2_TeamID  T2_FGMmean  T2_FGAmean  T2_FGM3mean  T2_FGA3mean  \
0    2003       1102   19.142857   39.785714     7.821429    20.821429   
1    2003       1103   27.148148   55.851852     5.444444    16.074074   
2    2003       1104   24.035714   57.178571     6.357143    19.857143   
3    2003       1105   24.384615   61.615385     7.576923    20.769231   
4    2003       1106   23.428571   55.285714     6.107143    17.642857   

   T2_ORmean  T2_Astmean  T2_TOmean  T2_Stlmean  T2_PFmean  \
0   4.178571   13.000000  11.428571    5.964286  18.750000   
1   9.777778   15.222222  12.629630    7.259259  19.851852   
2  13.571429   12.107143  13.285714    6.607143  18.035714   
3  13.500000   14.538462  18.653846    9.307692  20.230769   
4  12.285714   11.678571  17.035714    8.357143  18.178571   

   T2_opponent_FGMmean  T2_opponent_FGAmean  T2_opponent_FGM3mean  \
0            19.285714            42.428571              4.750000   
1            27.777778            57.000000              6.666667   
2            23.250000            55.500000              6.357143   
3            27.000000            58.961538              6.269231   
4            21.714286            53.392857              4.785714   

   T2_opponent_FGA3mean  T2_opponent_ORmean  T2_opponent_Astmean  \
0             12.428571            9.607143             9.142857   
1             18.370370           12.037037            15.481481   
2             19.142857           10.892857            11.678571   
3             17.538462           13.192308            15.807692   
4             15.214286           11.321429            11.785714   

   T2_opponent_TOmean  T2_opponent_Stlmean  T2_opponent_Blkmean  \
0           12.964286             5.428571             1.571429   
1           15.333333             6.407407             2.851852   
2           13.857143             5.535714             3.178571   
3           18.807692             9.384615             4.192308   
4           15.071429             8.785714             3.178571   

   T2_PointDiffmean  
0          0.250000  
1          0.629630  
2          4.285714  
3         -4.884615  
4         -0.142857

In [29]:
season_statistics_T2 = season_statistics_T2[season_statistics_T2['Season'] >= 2015]
season_statistics_T1 = season_statistics_T1[season_statistics_T1['Season'] >= 2015]

In [30]:
mteams_df = pd.read_csv('MTeams.csv')
mteams_df

TeamID          TeamName  FirstD1Season  LastD1Season
0      1101       Abilene Chr           2014          2024
1      1102         Air Force           1985          2024
2      1103             Akron           1985          2024
3      1104           Alabama           1985          2024
4      1105       Alabama A&M           2000          2024
..      ...               ...            ...           ...
373    1474         Queens NC           2023          2024
374    1475  Southern Indiana           2023          2024
375    1476         Stonehill           2023          2024
376    1477   TX A&M Commerce           2023          2024
377    1478          Le Moyne           2024          2024

[378 rows x 4 columns]

In [31]:
merged_df_T2 = pd.merge(season_statistics_T2, mteams_df, left_on='T2_TeamID', right_on='TeamID')

In [32]:
merged_df_T1 = pd.merge(season_statistics_T1, mteams_df, left_on='T1_TeamID', right_on='TeamID')
merged_df_T1

Season  T1_TeamID  T1_FGMmean  T1_FGAmean  T1_FGM3mean  T1_FGA3mean  \
0       2015       1101   21.428571   52.928571     7.250000    19.178571   
1       2016       1101   23.814815   53.925926     6.851852    18.851852   
2       2017       1101   24.440000   53.320000     7.080000    19.080000   
3       2018       1101   25.518519   56.481481     6.481481    19.962963   
4       2019       1101   25.344828   55.241379     7.241379    18.896552   
...      ...        ...         ...         ...          ...          ...   
3535    2023       1476   22.900000   52.433333     7.533333    21.633333   
3536    2024       1476   22.933333   57.933333     8.200000    27.200000   
3537    2023       1477   25.593750   56.687500     8.250000    25.843750   
3538    2024       1477   23.300000   59.400000     8.033333    27.333333   
3539    2024       1478   24.586207   58.517241     9.482759    27.793103   

      T1_ORmean  T1_Astmean  T1_TOmean  T1_Stlmean  T1_PFmean  \
0      8.250000   11.857143  12.821429    5.928571  20.607143   
1      8.185185   12.888889  13.407407    6.740741  22.370370   
2      7.560000   13.600000  14.480000    7.000000  22.160000   
3      9.037037   13.888889  14.407407    7.148148  21.037037   
4      9.068966   14.620690  11.655172    8.000000  19.137931   
...         ...         ...        ...         ...        ...   
3535   4.966667   11.900000  12.800000    6.966667  15.133333   
3536   6.866667   11.866667  12.966667    6.266667  15.466667   
3537   7.500000   13.843750  12.437500    5.562500  18.750000   
3538   7.500000   12.266667  11.000000    7.366667  17.666667   
3539   7.103448   14.931034  10.758621    7.275862  15.655172   

      T1_opponent_FGMmean  T1_opponent_FGAmean  T1_opponent_FGM3mean  \
0               25.035714            50.857143              6.178571   
1               26.481481            53.666667              5.592593   
2               24.840000            53.680000              7.080000   
3               24.777778            54.296296              6.629630   
4               22.482759            51.137931              5.724138   
...                   ...                  ...                   ...   
3535            25.366667            58.833333              8.300000   
3536            26.966667            60.700000             10.066667   
3537            26.093750            55.843750              6.781250   
3538            27.133333            57.600000              6.133333   
3539            25.689655            59.137931              8.482759   

      T1_opponent_FGA3mean  T1_opponent_ORmean  T1_opponent_Astmean  \
0                16.714286           10.035714            12.928571   
1                15.148148            8.592593            12.851852   
2                19.600000           10.640000            13.600000   
3                19.740741            9.925926            12.037037   
4                17.103448            8.827586            11.103448   
...                    ...                 ...                  ...   
3535             24.000000            9.566667            13.900000   
3536             28.033333            9.766667            16.400000   
3537             19.500000            8.125000            13.031250   
3538             17.500000            9.466667            13.833333   
3539             25.413793            9.344828            14.448276   

      T1_opponent_TOmean  T1_opponent_Stlmean  T1_opponent_Blkmean  \
0              13.464286             5.857143             4.250000   
1              13.444444             5.111111             3.296296   
2              14.800000             6.040000             3.160000   
3              15.481481             6.518519             2.925926   
4              15.724138             5.551724             2.965517   
...                  ...                  ...                  ...   
3535           12.900000             6.966667             2.933333   
3536           11.1666

Scrape Try

In [33]:
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
import requests
import time
import requests

In [34]:

base_urls=[
          'https://web.archive.org/web/20110311233233/http://www.kenpom.com/',
          'https://web.archive.org/web/20120311165019/http://kenpom.com/',
          'https://web.archive.org/web/20130318221134/http://kenpom.com/',
          'https://web.archive.org/web/20140318100454/http://kenpom.com/',
          'https://web.archive.org/web/20150316212936/http://kenpom.com/',
          'https://web.archive.org/web/20160314134726/http://kenpom.com/',
          'https://web.archive.org/web/20170312131016/http://kenpom.com/',
          'https://web.archive.org/web/20180311122559/https://kenpom.com/',
          'https://web.archive.org/web/20190317211809/https://kenpom.com/',
    'https://web.archive.org/web/20200319000000/https://kenpom.com/index.php?y=2020', # Adjust the timestamp accordingly
    'https://web.archive.org/web/20210319000000/https://kenpom.com/index.php?y=2021', # Adjust the timestamp accordingly
    'https://web.archive.org/web/20220319000000/https://kenpom.com/index.php?y=2022', # Adjust the timestamp accordingly
    'https://web.archive.org/web/20230319000000/https://kenpom.com/index.php?y=2023', # Adjust the timestamp accordingly
    'https://web.archive.org/web/20240319000000/https://kenpom.com/index.php', #
         ]


def scrap_archive(url, year):
    """
    Imports raw data from a kenpom archive into a dataframe
    """
    retries = 3
    delay = 1  # Delay in seconds between retries
    for _ in range(retries):
        try:
            headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
            page = requests.get(url, headers=headers)
            soup = BeautifulSoup(page.text, 'html.parser')
            table_full = soup.find_all('table', {'id': 'ratings-table'})

            thead = table_full[0].find_all('thead')
            table = table_full[0]

            for weird in thead:
                table = str(table).replace(str(weird), '')

            df = pd.read_html(table)[0]
            df['Year'] = year
            return df
        except requests.ConnectionError as e:
            print(f"Connection error occurred: {e}")
            print(f"Retrying in {delay} seconds...")
            time.sleep(delay)
            continue
    else:
        # If all retries fail, raise the last exception
        Exception("Failed to connect after multiple retries")


In [35]:
def scraping(df, year):
    for url in base_urls:
        print(f'Scraping: {url}')
        archive = scrap_archive(url, year)
        df = pd.concat((df, archive), axis=0)
        year += 1
    
    df.columns = ['Rank', 'Team', 'Conference', 'W-L', 'Pyth', 
             'AdjustO', 'AdjustO Rank', 'AdjustD', 'AdjustD Rank',
             'AdjustT', 'AdjustT Rank', 'Luck', 'Luck Rank', 
             'SOS Pyth', 'SOS Pyth Rank', 'SOS OppO', 'SOS OppO Rank',
             'SOS OppD', 'SOS OppD Rank', 'NCSOS Pyth', 'NCSOS Pyth Rank', 'Year']
    
    df = df[['Year', 'Team', 'AdjustO', 'AdjustD', 'Luck', 'Rank', 'Pyth']]
    df.columns = ['Season', 'TeamName', 'adj_o', 'adj_d', 'luck', 'rank', 'pyth']
    df['TeamName'] = df['TeamName'].apply(lambda x: re.sub('\d', '', x).strip()).replace('.','')
    return df

In [36]:
df=None
year=2011
df=scraping(df,year)
df.head()

Scraping: https://web.archive.org/web/20110311233233/http://www.kenpom.com/
Scraping: https://web.archive.org/web/20120311165019/http://kenpom.com/
Scraping: https://web.archive.org/web/20130318221134/http://kenpom.com/
Scraping: https://web.archive.org/web/20140318100454/http://kenpom.com/
Scraping: https://web.archive.org/web/20150316212936/http://kenpom.com/
Scraping: https://web.archive.org/web/20160314134726/http://kenpom.com/
Scraping: https://web.archive.org/web/20170312131016/http://kenpom.com/
Connection error occurred: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20170312131016/http://kenpom.com/ (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x17e7d1e90>: Failed to establish a new connection: [Errno 61] Connection refused'))
Retrying in 1 seconds...
Connection error occurred: HTTPSConnectionPool(host='web.archive.org', port=443): Max retries exceeded with url: /web/20170312131016/http://kenpom.com/ (

KeyboardInterrupt: 

In [ ]:
df_KP = df.copy()
df_KP

In [ ]:
df_KP.TeamName=df_KP.TeamName.apply(lambda x: x.replace('-',' '))
df_KP.TeamName=df_KP.TeamName.apply(lambda x: x.lower())
df_KP.TeamName=df_KP.TeamName.apply(lambda x: x.strip())
df_KP.TeamName=df_KP.TeamName.replace('mississippi valley st.','mississippi valley state')
#df.TeamName=df.TeamName.replace('texas a&m corpus chris','texas a&m corpus christi')
df_KP.TeamName=df_KP.TeamName.replace('dixie st.','dixie st')
df_KP.TeamName=df_KP.TeamName.replace('st. francis pa','st francis pa')
df_KP.TeamName=df_KP.TeamName.replace('ut rio grande valley','texas rio grande valley')
df_KP.TeamName=df_KP.TeamName.replace('southeast missouri st.','southeast missouri state')
df_KP.TeamName=df_KP.TeamName.replace('tarleton st.','tarleton st')
df_KP.TeamName=df_KP.TeamName.replace('liu','liu brooklyn')
df_KP.TeamName=df_KP.TeamName.replace('cal st. bakersfield','cal state bakersfield')

df_KP.TeamName=df_KP.TeamName.replace('virginia military inst','virginia military	')
df_KP.TeamName=df_KP.TeamName.replace('louisiana saint','louisiana state')
df_KP.TeamName=df_KP.TeamName.replace('nj inst of technology','njit')

df_KP.TeamName=df_KP.TeamName.replace('texas a&m corpus chris','texas a&m corpus')
df_KP.TeamName=df_KP.TeamName.replace('md baltimore county','maryland baltimore county')
#-------------------------------------------------------
# merge with spelling file to get the TeamID
spelling=pd.read_csv('MTeamSpellings.csv',encoding='cp1252')
spelling.columns=['TeamName','TeamID']
spelling.TeamName=spelling.TeamName.apply(lambda x: x.replace('-',' '))
df_KP.TeamName=df_KP.TeamName.apply(lambda x: x.strip())


df_KP=df_KP.merge(spelling[['TeamName','TeamID']],on='TeamName',how='left')

df_KP.TeamName=df_KP.TeamName.apply(lambda x: x.replace('st.','saint'))
df_KP.TeamName=df_KP.TeamName.apply(lambda x: x.replace(';',''))
df_KP.TeamName=df_KP.TeamName.apply(lambda x: x.replace('\t',''))
df_KP.TeamName=df_KP.TeamName.replace('texas a&m corpus chris','texas a&m corpus')
df_KP.TeamName=df_KP.TeamName.replace('louisiana saint','louisiana state')

df_KP=df_KP.merge(spelling[['TeamName','TeamID']],on='TeamName',how='left')

df_KP.TeamID_x.fillna(df_KP.TeamID_y, inplace=True)

df_KP=df_KP[['Season','TeamID_x','adj_o','adj_d','luck','rank','pyth']]
df_KP.columns=['Season','TeamID','adj_o','adj_d','luck','rank','pyth']
df_KP['TeamID'] = df_KP['TeamID'].astype('Int64')

In [ ]:
df_KP.columns=['Season','TeamID','adj_o','adj_d','luck','rank','adj_em']
df_KP

In [ ]:
season_statistics_T1 = pd.merge(season_statistics_T1, df_KP, left_on=['Season', 'T1_TeamID'], right_on=['Season', 'TeamID'], how='left')
season_statistics_T2 = pd.merge(season_statistics_T2, df_KP, left_on=['Season', 'T2_TeamID'], right_on=['Season', 'TeamID'], how='left')
season_statistics_T2.columns

In [ ]:
# Selecting and renaming columns for season_statistics_T1
selected_columns_T1 = ["Season", "TeamID", "adj_o", "adj_d", "luck", "rank", "adj_em"]
renamed_columns_T1 = ["T1_" + x if x not in ["Season", "TeamID"] else x for x in selected_columns_T1]
season_statistics_T1 = season_statistics_T1[selected_columns_T1].rename(columns=dict(zip(selected_columns_T1, renamed_columns_T1)))

# Selecting and renaming columns for season_statistics_T2
selected_columns_T2 = ["Season", "TeamID", "adj_o", "adj_d", "luck", "rank", "adj_em"]
renamed_columns_T2 = ["T2_" + x if x not in ["Season", "TeamID"] else x for x in selected_columns_T2]
season_statistics_T2 = season_statistics_T2[selected_columns_T2].rename(columns=dict(zip(selected_columns_T2, renamed_columns_T2)))

# Drop 'Season' and 'TeamID' columns from both dataframes
season_statistics_T1.drop(columns=["TeamID"], inplace=True)
season_statistics_T2.drop(columns=["TeamID"], inplace=True)

# Concatenate the original dataframes with the modified ones
season_statistics_T1 = pd.concat([season_statistics_T1, season_statistics_T2], axis=1)
season_statistics_T1

In [ ]:
season_statistics_T1.drop(season_statistics_T1.columns[6], axis=1, inplace=True)

In [ ]:
season_statistics_T1

# About to Merge with Tourney Data

In [ ]:
tourney_data = tourney_data[['Season', 'DayNum', 'T1_TeamID', 'T1_Score', 'T2_TeamID' ,'T2_Score']]
tourney_data.head()

# Fix this below so that the Ken Pom stuff shows Up

In [ ]:
tourney_data = pd.merge(tourney_data, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')
tourney_data = tourney_data.drop(tourney_data.filter(regex='_x|_y').columns, axis=1)
tourney_data = tourney_data[tourney_data['Season'] >= 2015]
tourney_data.columns

In [ ]:
regular_season_effects = regular_data[['Season','T1_TeamID','T2_TeamID','PointDiff']].copy()
regular_season_effects['T1_TeamID'] = regular_season_effects['T1_TeamID'].astype(str)
regular_season_effects['T2_TeamID'] = regular_season_effects['T2_TeamID'].astype(str)
regular_season_effects['win'] = np.where(regular_season_effects['PointDiff']>0,1,0)
march_madness = pd.merge(seeds[['Season','TeamID']],seeds[['Season','TeamID']],on='Season')
march_madness.columns = ['Season', 'T1_TeamID', 'T2_TeamID']
march_madness.T1_TeamID = march_madness.T1_TeamID.astype(str)
march_madness.T2_TeamID = march_madness.T2_TeamID.astype(str)
regular_season_effects = pd.merge(regular_season_effects, march_madness, on = ['Season','T1_TeamID','T2_TeamID'])
regular_season_effects.shape

In [ ]:
regular_season_effects.head()

# Get the Seeds so we can put in model

In [ ]:
seeds.head()

In [ ]:
seeds['seed'] = seeds['Seed'].apply(lambda x: int(x[1:3]))
seeds.tail()

In [ ]:
seeds_T1 = seeds[['Season','TeamID','seed']].copy()
seeds_T2 = seeds[['Season','TeamID','seed']].copy()
seeds_T1.columns = ['Season','T1_TeamID','T1_seed']
seeds_T2.columns = ['Season','T2_TeamID','T2_seed']

In [ ]:
tourney_data = pd.merge(tourney_data, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
tourney_data = pd.merge(tourney_data, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')

get a seed difference for model 

In [ ]:
tourney_data["Seed_diff"] = tourney_data["T1_seed"] - tourney_data["T2_seed"]

In [ ]:
y = tourney_data['T1_Score'] - tourney_data['T2_Score']
y.describe()

In [ ]:
tourney_data

## See all our features for Model

In [ ]:
features = list(season_statistics_T1.columns[2:999]) + \
    list(season_statistics_T2.columns[2:999]) + \
    list(seeds_T1.columns[2:999]) + \
    list(seeds_T2.columns[2:999]) + \
    ["Seed_diff"] + 

features

In [ ]:
X = tourney_data[features].values
dtrain = xgb.DMatrix(X, label = y)

In [ ]:
def cauchyobj(preds, dtrain):
    labels = dtrain.get_label()
    c = 5000 
    x =  preds-labels    
    grad = x / (x**2/c**2+1)
    hess = -c**2*(x**2-c**2)/(x**2+c**2)**2
    return grad, hess

In [ ]:
param = {} 
#param['objective'] = 'reg:linear'
param['eval_metric'] =  'mae'
param['booster'] = 'gbtree'
param['eta'] = 0.05 #change to ~0.02 for final run
param['subsample'] = 0.35
param['colsample_bytree'] = 0.7
param['num_parallel_tree'] = 10 #recommend 10
param['min_child_weight'] = 40
param['gamma'] = 10
param['max_depth'] =  3
param['silent'] = 1

print(param)

In [ ]:
xgb_cv = []
repeat_cv = 3 # recommend 10

for i in range(repeat_cv): 
    print(f"Fold repeater {i}")
    xgb_cv.append(
        xgb.cv(
          params = param,
          dtrain = dtrain,
          obj = cauchyobj,
          num_boost_round = 3000,
          folds = KFold(n_splits = 5, shuffle = True, random_state = i),
          early_stopping_rounds = 25,
          verbose_eval = 50
        )
    )


In [ ]:
iteration_counts = [np.argmin(x['test-mae-mean'].values) for x in xgb_cv]
val_mae = [np.min(x['test-mae-mean'].values) for x in xgb_cv]
iteration_counts, val_mae

In [ ]:
oof_preds = []
for i in range(repeat_cv):
    print(f"Fold repeater {i}")
    preds = y.copy()
    kfold = KFold(n_splits = 5, shuffle = True, random_state = i)    
    for train_index, val_index in kfold.split(X,y):
        dtrain_i = xgb.DMatrix(X[train_index], label = y[train_index])
        dval_i = xgb.DMatrix(X[val_index], label = y[val_index])  
        model = xgb.train(
              params = param,
              dtrain = dtrain_i,
              num_boost_round = iteration_counts[i],
              verbose_eval = 50
        )
        preds[val_index] = model.predict(dval_i)
    oof_preds.append(np.clip(preds,-30,30))

In [ ]:
plot_df = pd.DataFrame({"pred":oof_preds[0], "label":np.where(y>0,1,0)})
plot_df["pred_int"] = plot_df["pred"].astype(int)
plot_df = plot_df.groupby('pred_int')['label'].mean().reset_index(name='average_win_pct')

plt.figure()
plt.plot(plot_df.pred_int,plot_df.average_win_pct)

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
        
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    
    print(f"logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16)] = 1.0
    spline_fit[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15)] = 1.0
    spline_fit[(tourney_data.T1_seed==3) & (tourney_data.T2_seed==14)] = 1.0
    spline_fit[(tourney_data.T1_seed==4) & (tourney_data.T2_seed==13)] = 1.0
    spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1)] = 0.0
    spline_fit[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2)] = 0.0
    spline_fit[(tourney_data.T1_seed==14) & (tourney_data.T2_seed==3)] = 0.0
    spline_fit[(tourney_data.T1_seed==13) & (tourney_data.T2_seed==4)] = 0.0
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
pd.concat(
    [tourney_data[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score < tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15) & (tourney_data.T1_Score < tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==3) & (tourney_data.T2_seed==14) & (tourney_data.T1_Score < tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==4) & (tourney_data.T2_seed==13) & (tourney_data.T1_Score < tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score > tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2) & (tourney_data.T1_Score > tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==14) & (tourney_data.T2_seed==3) & (tourney_data.T1_Score > tourney_data.T2_Score)],
     tourney_data[(tourney_data.T1_seed==13) & (tourney_data.T2_seed==4) & (tourney_data.T1_Score > tourney_data.T2_Score)]]
)   

In [ ]:
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==3) & (tourney_data.T2_seed==14) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==4) & (tourney_data.T2_seed==13) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    spline_fit[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    spline_fit[(tourney_data.T1_seed==14) & (tourney_data.T2_seed==3) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    spline_fit[(tourney_data.T1_seed==13) & (tourney_data.T2_seed==4) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 

In [ ]:
val_cv = []
spline_model = []

for i in range(repeat_cv):
    dat = list(zip(oof_preds[i],np.where(y>0,1,0)))
    dat = sorted(dat, key = lambda x: x[0])
    datdict = {}
    for k in range(len(dat)):
        datdict[dat[k][0]]= dat[k][1]
    spline_model.append(UnivariateSpline(list(datdict.keys()), list(datdict.values())))
    spline_fit = spline_model[i](oof_preds[i])
    spline_fit = np.clip(spline_fit,0.025,0.975)
    spline_fit[(tourney_data.T1_seed==1) & (tourney_data.T2_seed==16) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==2) & (tourney_data.T2_seed==15) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==3) & (tourney_data.T2_seed==14) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==4) & (tourney_data.T2_seed==13) & (tourney_data.T1_Score > tourney_data.T2_Score)] = 1.0
    spline_fit[(tourney_data.T1_seed==16) & (tourney_data.T2_seed==1) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    spline_fit[(tourney_data.T1_seed==15) & (tourney_data.T2_seed==2) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    spline_fit[(tourney_data.T1_seed==14) & (tourney_data.T2_seed==3) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    spline_fit[(tourney_data.T1_seed==13) & (tourney_data.T2_seed==4) & (tourney_data.T1_Score < tourney_data.T2_Score)] = 0.0
    
    val_cv.append(pd.DataFrame({"y":np.where(y>0,1,0), "pred":spline_fit, "season":tourney_data.Season}))
    print(f"adjusted logloss of cvsplit {i}: {log_loss(np.where(y>0,1,0),spline_fit)}") 
    
val_cv = pd.concat(val_cv)
val_cv.groupby('season').apply(lambda x: log_loss(x.y, x.pred))

# Do Submission formatting Below This 

In [ ]:
sub = pd.read_csv("SampleSubmission2023.csv")
sub['Season'] = sub['ID'].apply(lambda x: int(x.split('_')[0]))
sub["T1_TeamID"] = sub['ID'].apply(lambda x: int(x.split('_')[1]))
sub["T2_TeamID"] = sub['ID'].apply(lambda x: int(x.split('_')[2]))
sub.head()

In [ ]:
seeds_T1.tail()

In [ ]:
sub = pd.merge(sub, season_statistics_T1, on = ['Season', 'T1_TeamID'], how = 'left')
sub = pd.merge(sub, season_statistics_T2, on = ['Season', 'T2_TeamID'], how = 'left')


sub = pd.merge(sub, seeds_T1, on = ['Season', 'T1_TeamID'], how = 'left')
sub = pd.merge(sub, seeds_T2, on = ['Season', 'T2_TeamID'], how = 'left')


sub["Seed_diff"] = sub["T1_seed"] - sub["T2_seed"]

sub.head()

In [ ]:
Xsub = sub[features].values
dtest = xgb.DMatrix(Xsub)

In [ ]:
sub_models = []
for i in range(repeat_cv):
    print(f"Fold repeater {i}")
    sub_models.append(
        xgb.train(
          params = param,
          dtrain = dtrain,
          num_boost_round = int(iteration_counts[i] * 1.05),
          verbose_eval = 50
        )
    )

In [ ]:
sub_preds = []
for i in range(repeat_cv):
    sub_preds.append(np.clip(spline_model[i](np.clip(sub_models[i].predict(dtest),-30,30)),0.025,0.975))
    
sub["Pred"] = pd.DataFrame(sub_preds).mean(axis=0)
"""
sub.loc[(sub.T1_seed==1) & (sub.T2_seed==16), 'Pred'] = 1.0
sub.loc[(sub.T1_seed==2) & (sub.T2_seed==15), 'Pred'] = 1.0
sub.loc[(sub.T1_seed==3) & (sub.T2_seed==14), 'Pred'] = 1.0
sub.loc[(sub.T1_seed==4) & (sub.T2_seed==13), 'Pred'] = 1.0
sub.loc[(sub.T1_seed==16) & (sub.T2_seed==1), 'Pred'] = 0.0
sub.loc[(sub.T1_seed==15) & (sub.T2_seed==2), 'Pred'] = 0.0
sub.loc[(sub.T1_seed==14) & (sub.T2_seed==3), 'Pred'] = 0.0
sub.loc[(sub.T1_seed==13) & (sub.T2_seed==4), 'Pred'] = 0.0
"""
sub[['ID','Pred']].to_csv("submission.csv", index = None)

In [ ]:
sub.head()

In [ ]:
round_slots = pd.read_csv('MNCAATourneySlots.csv')
round_slots = round_slots[round_slots['Season'] == 2023]
round_slots = round_slots[round_slots['Slot'].str.contains('R')] # Filter out First Four

seeds = pd.read_csv('2024_tourney_seeds.csv')
seeds_m = seeds[seeds['Tournament'] == 'M']
seeds_w = seeds[seeds['Tournament'] == 'W']

# Predictions of last year's 1st place solution by RustyB: https://www.kaggle.com/code/rustyb/paris-madness-2023/output
preds = pd.read_csv('submission.csv') 
preds['ID'] = preds['ID'].str.split('_')

In [ ]:
from tqdm import tqdm
def prepare_data(seeds, preds):
    # Function preparing the data for the simulation
    seed_dict = seeds.set_index('Seed')['TeamID'].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}
    probas_dict = {}
    
    for teams, proba in zip(preds['ID'], preds['Pred']):
        team1, team2 = teams[1], teams[2]

        probas_dict.setdefault(team1, {})[team2] = proba
        probas_dict.setdefault(team2, {})[team1] = 1 - proba

    return seed_dict, inverted_seed_dict, probas_dict


def simulate(round_slots, seeds, inverted_seeds, probas, random_values, sim=True):
    '''
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - probas (dict): Dictionary containing matchup probabilities.
    - random_values (array-like): Array with precomputed random-values.
    - sim (boolean): Simulates match if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    '''
    winners = []
    slots = []

    for slot, strong, weak, random_val in zip(round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed, random_values):
        team1, team2 = seeds[strong], seeds[weak]

        # Get the probability of team_1 winning
        proba = probas[str(team1)][str(team2)]
            
        if sim:
            # Randomly determine the winner based on the probability
            winner = team1 if random_val < proba else team2
        else:
            # Determine the winner based on the higher probability
            winner = [team1, team2][np.argmax([proba, 1-proba])]
            
        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    # Convert winners to original seeds using the inverted_seeds dictionary
    return [inverted_seeds[w] for w in winners], slots


def run_simulation(brackets=1, seeds=None, preds=None, round_slots=None, sim=True):
    '''
    Runs a simulation of bracket tournaments.

    Parameters:
    - brackets (int): Number of brackets to simulate.
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - preds (pd.DataFrame): DataFrame containing prediction information for each match-up.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - sim (boolean): Simulates matches if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    '''
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict, probas_dict = prepare_data(seeds, preds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []
    
    # Precompute random-values
    random_values = np.random.random(size=(brackets, len(round_slots)))

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets+1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probas_dict, random_values[b-1], sim)
        
        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({'Bracket': bracket, 'Slot': slots, 'Team': results})

    return result_df

n_brackets = 1000
result_m=run_simulation(brackets=n_brackets, seeds=seeds_m, preds=preds, round_slots=round_slots, sim=True)
result_m['Tournament'] = 'M'
result_w=run_simulation(brackets=n_brackets, seeds=seeds_w, preds=preds, round_slots=round_slots, sim=True)
result_w['Tournament'] = 'W'
submission = pd.concat([result_m, result_w])
submission.reset_index(inplace=True, drop=True)
submission.index.names = ['RowId']

In [ ]:
# submission.to_csv('submission.csv')
submission = submission[['Tournament', 'Bracket', 'Slot', 'Team']]
submission